In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import uncertainties as un
from matplotlib import pyplot as plt
from scipy.stats import t
from uncertainties import unumpy as unp

from funcs.simulation import thermo

In [69]:
df = pd.read_hdf("../simulation_measurement_comparison/simulated_and_measured.h5", key="data_fixed_uncert")
print(f"loaded data from {len(df)} shots")
df.head()

loaded data from 17 shots


,diluent,phi_nom,dil_mf_nom,p_0,u_p_0,t_0,phi,u_phi,dil_mf,u_dil_mf,wave_speed,u_wave_speed,cell_size_measured,u_cell_size_measured,cell_size_gavrikov,cell_size_ng,cell_size_westbrook
0,CO2,0.4,0.10,101393.842884,2.307000,289.846188,0.419182,0.000208,0.103943,0.000040,1780.064483,1.212763,17.711596,2.065540,770.810052,74.459337,40.577703
1,CO2,0.4,0.15,101400.200470,1.609842,290.021614,0.416662,0.000215,0.154795,0.000036,1742.324854,1.161985,26.122865,2.063038,2328.318780,137.733125,76.176883
2,CO2,0.4,0.20,101383.316513,1.976709,290.005443,0.425508,0.000378,0.207642,0.000063,1680.937965,1.326527,45.265852,7.552310,7173.643295,262.627217,148.643042
3,CO2,0.7,0.10,101393.617108,2.121143,289.767941,0.735369,0.000243,0.104023,0.000045,1944.385714,1.446872,11.744907,2.280838,275.832035,41.773144,24.192077
4,CO2,0.7,0.15,101412.528953,2.179424,289.308769,0.730354,0.000250,0.154600,0.000045,1884.805654,1.665023,14.199823,6.516796,777.986099,72.212132,43.211558


In [72]:
if "dil_mf_co2e" not in df.columns:
    df.insert(len(df.columns), "dil_mf_co2e", np.NaN)
if "u_dil_mf_co2e" not in df.columns:
    df.insert(len(df.columns), "u_dil_mf_co2e", np.NaN)

for idx, row in df.iterrows():
    if row.diluent == "CO2":
        df.iloc[idx, df.columns.get_loc("dil_mf_co2e")] = row.dil_mf
    else:
        df.iloc[idx, df.columns.get_loc("dil_mf_co2e")] = thermo.match_adiabatic_temp(
            mech="gri30_highT.xml",
            fuel="CH4",
            oxidizer="N2O",
            phi=row.phi,
            dil_original=row.diluent,
            dil_original_mol_frac=row.dil_mf,
            dil_new="CO2",
            init_temp=row.t_0,
            init_press=row.p_0,
        )
    df.iloc[idx, df.columns.get_loc("u_dil_mf_co2e")] = row.u_dil_mf

df

,diluent,phi_nom,dil_mf_nom,p_0,u_p_0,t_0,phi,u_phi,dil_mf,u_dil_mf,wave_speed,u_wave_speed,cell_size_measured,u_cell_size_measured,cell_size_gavrikov,cell_size_ng,cell_size_westbrook,dil_mf_co2e,u_dil_mf_co2e
0,CO2,0.4,0.10,101393.842884,2.307000,289.846188,0.419182,0.000208,0.103943,0.000040,1780.064483,1.212763,17.711596,2.065540,770.810052,74.459337,40.577703,0.103943,0.000040
1,CO2,0.4,0.15,101400.200470,1.609842,290.021614,0.416662,0.000215,0.154795,0.000036,1742.324854,1.161985,26.122865,2.063038,2328.318780,137.733125,76.176883,0.154795,0.000036
2,CO2,0.4,0.20,101383.316513,1.976709,290.005443,0.425508,0.000378,0.207642,0.000063,1680.937965,1.326527,45.265852,7.552310,7173.643295,262.627217,148.643042,0.207642,0.000063
3,CO2,0.7,0.10,101393.617108,2.121143,289.767941,0.735369,0.000243,0.104023,0.000045,1944.385714,1.446872,11.744907,2.280838,275.832035,41.773144,24.192077,0.104023,0.000045
4,CO2,0.7,0.15,101412.528953,2.179424,289.308769,0.730354,0.000250,0.154600,0.000045,1884.805654,1.665023,14.199823,6.516796,777.986099,72.212132,43.211558,0.154600,0.000045
5,CO2,0.7,0.20,101412.244820,1.938748,290.890143,0.739247,0.000282,0.207012,0.000045,1808.159887,1.251217,22.565446,2.059861,2342.272709,127.963220,79.797353,0.207012,0.000045
6,CO2,1.0,0.10,101396.288279,2.719599,290.170199,1.048230,0.000235,0.103722,0.000039,2030.771107,1.578345,10.729874,1.757072,213.161962,33.742735,19.590121,0.103722,0.000039
7,CO2,1.0,0.15,101404.272743,1.800340,290.141360,1.048723,0.000231,0.154530,0.000037,1965.920228,1.479180,13.662187,2.065471,668.234272,56.702087,34.275813,0.154530,0.000037
8,CO2,1.0,0.20,101410.128009,1.557379,290.441929,1.052847,0.000288,0.206571,0.000042,1907.275986,1.392106,18.601504,2.084832,2103.742085,98.650499,62.648736,0.206571,0.000042
9,N2,0.4,0.10,101368.606788,2.121188,289.359526,0.416727,0.000224,0.180426,0.000041,1880.070432,1.657319,15.263125,7.642898,275.935516,47.345764,24.757695,0.102732,0.000041


In [73]:
df.to_hdf("../simulation_measurement_comparison/simulated_and_measured.h5", key="data_fixed_uncert_with_co2e")

In [76]:
df = pd.read_csv("../cj_study/cj_tad_ss_results.csv")

In [77]:
if "dil_mf_co2e" not in df.columns:
    df.insert(len(df.columns), "dil_mf_co2e", np.NaN)
if "u_dil_mf_co2e" not in df.columns:
    df.insert(len(df.columns), "u_dil_mf_co2e", np.NaN)

for idx, row in df.iterrows():
    if row.diluent == "CO2":
        df.iloc[idx, df.columns.get_loc("dil_mf_co2e")] = row.dil_mf
    else:
        df.iloc[idx, df.columns.get_loc("dil_mf_co2e")] = thermo.match_adiabatic_temp(
            mech="gri30_highT.xml",
            fuel="CH4",
            oxidizer="N2O",
            phi=row.phi,
            dil_original=row.diluent,
            dil_original_mol_frac=row.dil_mf,
            dil_new="CO2",
            init_temp=row.t_0,
            init_press=row.p_0,
        )
    df.iloc[idx, df.columns.get_loc("u_dil_mf_co2e")] = row.u_dil_mf

df

,date,shot,fuel,oxidizer,diluent,t_0,u_t_0,p_0_nom,p_0,u_p_0,...,dil_mf_nom,dil_mf,u_dil_mf,wave_speed,u_wave_speed,t_ad,cj_speed,sound_speed,dil_mf_co2e,u_dil_mf_co2e
0,2020-11-23,0,CH4,N2O,N2,288.854426,3.487385,101325.0,101321.386226,3.933011,...,0.10,0.232712,0.000099,2059.459459,2.811308,2794.637781,2114.464151,1148.785328,0.105727,0.000099
1,2020-12-10,0,CH4,N2O,N2,288.249476,3.487385,101325.0,101441.621314,3.951862,...,0.15,0.250255,0.000054,1795.959596,2.137938,2436.412954,1895.341221,1041.095726,0.154159,0.000054
2,2020-11-25,0,CH4,N2O,N2,289.117440,3.487385,101325.0,101312.656033,3.990513,...,0.20,0.387742,0.000101,1961.029412,2.549002,2657.521685,2031.781270,1108.463881,0.212989,0.000101
3,2020-12-09,0,CH4,N2O,CO2,288.718362,3.487385,101325.0,101432.336834,3.883890,...,0.10,0.102603,0.000052,1783.946488,2.109433,2516.543411,1875.787281,1021.580802,0.102603,0.000052
4,2020-11-12,0,CH4,N2O,N2,290.323806,3.493193,101325.0,101327.131884,4.019152,...,0.20,0.386997,0.000086,1979.220779,2.596513,2658.704224,2033.211722,1109.304497,0.212536,0.000086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,2020-07-16,4,C3H8,air,N2,302.566033,3.487385,101325.0,101409.398282,3.726218,...,0.01,0.017138,0.000049,1786.934673,2.116505,2135.684856,1751.407884,973.250853,0.006331,0.000049
525,2020-07-16,5,C3H8,air,N2,303.230994,3.487385,101325.0,101396.287348,3.446007,...,0.01,0.017146,0.000048,1682.649842,1.876677,2144.033392,1753.963240,974.302378,0.006327,0.000048
526,2020-07-16,6,C3H8,air,N2,303.494991,3.487385,101325.0,101395.528971,2.999971,...,0.01,0.016905,0.000049,1693.333333,1.900583,2148.421126,1755.329020,974.864496,0.006233,0.000049
527,2020-07-16,8,C3H8,air,N2,304.572489,3.493193,101325.0,101383.889694,2.573943,...,0.01,0.017055,0.000049,1669.483568,1.847423,2136.932128,1751.305788,973.317669,0.006297,0.000049


In [78]:
df.to_csv("../cj_study/cj_tad_ss_results_with_co2e.csv")